In [1]:
import os
import glob
import numpy as np
import cv2
import json
from pprint import pprint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dense, merge
from keras.models import Model
from keras.layers import Convolution2D, MaxPooling2D, Reshape, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
data_dir = "datasets/hive_fast_driving/"
data_save_dir = "datasets/npy/"

Using TensorFlow backend.


In [2]:
image_paths = []
json_paths = []
for root, dirs, files in os.walk("datasets/hive_fast_driving/"):
    for file in files:
        if file.endswith("_cam-image_array_.jpg"):
            image_paths.append(os.path.join(root, file))
        if file.endswith(".json"):
            json_paths.append(os.path.join(root, file))
image_paths = np.array(image_paths)
json_paths = np.array(json_paths)
image_paths = np.sort(image_paths)
json_paths = np.sort(json_paths)

In [3]:
data = np.zeros((image_paths.shape[0], 120, 160, 3))
for i in range(image_paths.shape[0]):
    data[i] = cv2.imread(image_paths[0])

In [10]:
angle_labels = np.zeros((json_paths.shape[0], 1))
throttle_labels = np.zeros((json_paths.shape[0], 1))
labels =  np.zeros((json_paths.shape[0], 2))
for i in range(json_paths.shape[0]):
    json_data = json.load(open(json_paths[i]))
    angle_labels[i] = json_data["user/angle"]
    throttle_labels[i] = json_data["user/throttle"]
    labels[i][0] = angle_labels[i]
    labels[i][1] = throttle_labels[i]

In [5]:
#data_npy_path = "10113_donkey_data.npy"
#labels_npy_path = "10113_donkey_labels.npy"
#np.save(os.path.join(data_save_dir, data_npy_path), data)
#np.save(os.path.join(data_save_dir, labels_npy_path), labels)

In [34]:
def myCnn():
    img_in = Input(shape=(120, 160, 3), name='img_in')                      # First layer, input layer, Shape comes from camera.py resolution, RGB
    x = img_in
    x = Convolution2D(24, (5,5), strides=(2,2), activation='relu')(x)       # 24 features, 5 pixel x 5 pixel kernel (convolution, feauture) window, 2wx2h stride, relu activation
    x = Convolution2D(32, (5,5), strides=(2,2), activation='relu')(x)       # 32 features, 5px5p kernel window, 2wx2h stride, relu activatiion
    x = Convolution2D(64, (5,5), strides=(2,2), activation='relu')(x)       # 64 features, 5px5p kernal window, 2wx2h stride, relu
    x = Convolution2D(64, (3,3), strides=(2,2), activation='relu')(x)       # 64 features, 3px3p kernal window, 2wx2h stride, relu
    x = Convolution2D(64, (3,3), strides=(1,1), activation='relu')(x)       # 64 features, 3px3p kernal window, 1wx1h stride, relu

    # Possibly add MaxPooling (will make it less sensitive to position in image).  Camera angle fixed, so may not to be needed

    x = Flatten(name='flattened')(x)                                        # Flatten to 1D (Fully connected)
    x = Dense(100, activation='relu')(x)                                    # Classify the data into 100 features, make all negatives 0
    x = Dropout(.1)(x)                                                      # Randomly drop out (turn off) 10% of the neurons (Prevent overfitting)
    x = Dense(50, activation='relu')(x)                                     # Classify the data into 50 features, make all negatives 0
    x = Dropout(.1)(x)                                                      # Randomly drop out 10% of the neurons (Prevent overfitting)
    #categorical output of the angle
    angle_out = Dense(1, activation='sigmoid', name='angle_out')(x)        # Connect every input with every output and output 15 hidden units. Use Softmax to give percentage. 15 categories and find best one based off percentage 0.0-1.0
    
    #continous output of throttle
    throttle_out = Dense(1, activation='relu', name='throttle_out')(x)      # Reduce to 1 number, Positive number only
    
    model = Model(inputs=[img_in], outputs=[angle_out, throttle_out])
    model.compile(optimizer='adam',
                  loss={'angle_out': 'mean_absolute_error',
                        'throttle_out': 'mean_absolute_error'},
                  loss_weights={'angle_out': 0.9, 'throttle_out': .001})
    return model

In [32]:
model = myCnn()

In [33]:
model.fit(data, [angle_labels, throttle_labels], epochs=10)

Epoch 1/10
10115/10115 [==============================] - 28s 3ms/step - loss: 0.7969 - angle_out_loss: 0.8851 - throttle_out_loss: 0.3519
Epoch 2/10
10115/10115 [==============================] - 27s 3ms/step - loss: 0.7958 - angle_out_loss: 0.8839 - throttle_out_loss: 0.2840
Epoch 3/10
10115/10115 [==============================] - 28s 3ms/step - loss: 0.7958 - angle_out_loss: 0.8839 - throttle_out_loss: 0.2840
Epoch 4/10
 7936/10115 [======================>.......] - ETA: 6s - loss: 0.7961 - angle_out_loss: 0.8842 - throttle_out_loss: 0.2840

KeyboardInterrupt: 